# A notebook to make some scatter plots from PIPS data

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
# from pyCRMtools.pycaps import arps_read
# from pyCRMtools.pycaps import pycaps_fields
# from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
import matplotlib.colors as colors
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
import matplotlib.colors as colors
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

In [ ]:
# from natsort import natsorted
from pprint import pprint

def remove_unneeded(ds):
    dim_names_to_remove = ['fields_KHTX', 'fields_KGWX', 'fields']
    ds = ds.drop_dims(dim_names_to_remove, errors='ignore')
    return ds
#

# ND_tag = '_qc'
# rename_dict = {
#     'Dm43': 'Dm43_qc',
#     'sigma': 'sigma_qc',
#     'DSD_MM24': 'DSD_MM24_qc',
#     'DSD_MM36': 'DSD_MM36_qc',
#     'DSD_MM346': 'DSD_MM346_qc',
#     'DSD_MM246': 'DSD_MM246_qc',
#     'DSD_MM234': 'DSD_MM234_qc',
#     'DSD_TMM246': 'DSD_TMM246_qc',
# }

# rename_dict = {
#     ''
# }

PIPS_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/full_PIPS_dataset_RB15/'
parsivel_filenames = glob.glob(PIPS_dir + '/parsivel_combined*60s*nc')
# for parsivel_filename in parsivel_filenames[:1]:
#     parsivel_ds = xr.load_dataset(parsivel_filename)
# # #     varlist = [k for k, v in parsivel_ds.items()]
# # #     vars_to_rename = [var for var in varlist if any(x in var for x in ['_retr_SATP_RB15_qc'])]
# # #     new_varnames = [var.replace('SATP_RB15', 'SATP_TMM_RB15') for var in vars_to_rename]
# # #     rename_dict = {varname: newvarname for varname, newvarname in zip(vars_to_rename, new_varnames)}
# # #     # Rename some variables (one-time-only)
# # #     parsivel_ds = parsivel_ds.rename(rename_dict)
#     for retr_name in ['SATP_TMM', 'C08', 'Z01', 'TMM_F']:
#         varname_old = 'Dm_retr_{}{}'.format(retr_name, ND_tag)
#         varname_new = 'Dm43_retr_{}{}'.format(retr_name, ND_tag)
#         parsivel_ds = parsivel_ds.rename({varname_old: varname_new})
# # # #     # Next, remove all CG_coeff attributes
# # # #     attr_names = parsivel_ds.attrs.copy()
# # # #     for attr_name in attr_names:
# # # #         if 'CG_coeff' in attr_name:
# # # #             del parsivel_ds.attrs[attr_name]
# # # #     # Save back to file
#     parsivel_ds.to_netcdf(parsivel_filename)
    
    
#     print(parsivel_filename)
#     print(parsivel_ds['RR_retr_Z01_RB15_qc'])
    # print("Number of times in {} = {:d}".format(os.path.basename(parsivel_filename), parsivel_ds.sizes['time']))
parsivel_ds_read = xr.open_mfdataset(parsivel_filenames, combine='nested', concat_dim="time", 
                                     data_vars='minimal', coords='minimal', compat='override', parallel=True)
# parsivel_ds_read = xr.open_mfdataset(parsivel_filenames, combine='nested', concat_dim='time',
#                                      preprocess=pipsio.remove_unneeded) #, combine='nested', concat_dim='time')
# # parsivel_ds_read = xr.open_mfdataset(parsivel_filenames, combine='by_coords', concat_dim='time',
# #                                      preprocess=pipsio.remove_unneeded)

# print(parsivel_ds_read)

In [ ]:
plot_dir = os.path.join(PIPS_dir, 'plots/QC_vs_RB15_QC/')
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
figfmt = 'png'
# retr_names = ['SATP_TMM', 'C08', 'Z01', 'TMM_F']
# titles = ['SATP', 'C08', 'Z01', 'TMM (filtered)']

def plot_radar_field_one2one(varname1, varname2, var1, var2, col_field, title, axparams_var, 
                             deployment_name='Unknown', min_val=None, max_val=None):
    valid_indices = np.isfinite(var1) & np.isfinite(var2)
    var1 = var1.where(valid_indices)
    var2 = var2.where(valid_indices)
    var1_plt = var1
    var2_plt = var2
    if min_val:
        overlap = (var1_plt > min_val) & (var2_plt > min_val)
        var1_plt = var1_plt.where(overlap)
        var2_plt = var2_plt.where(overlap)
    if max_val:
        overlap = (var1_plt < max_val) & (var2_plt < max_val)
        var1_plt = var1_plt.where(overlap)
        var2_plt = var2_plt.where(overlap)
    var_ds = xr.Dataset({'var1': var1_plt, 'var2': var2_plt, 'col_field': col_field})
    fig, ax = PIPSplot.plot_one2one(var_ds, 'var1', 'var2', axparams_var)
    ax.set_title(title)
    plot_filename = '{}_{}_one2one_{}_at_{}.{}'.format(deployment_name, var1.name, varname1, 
                                                       varname2, figfmt)
    plot_path = os.path.join(plot_dir, plot_filename)
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')   

def plot_PIPS_retrieval_one2one(ds, varname, var_obs, titles, retr_names, ND_tag, col_field, filter_tag=''):
    for title, retr_name in zip(titles, retr_names):
        var_retr = parsivel_ds['{}_retr_{}{}'.format(varname, retr_name, ND_tag)]
        axparams_var = axparams[varname]
        overlap = (var_obs > 0.) & (var_retr > 0.)
        var_obs_plt = var_obs.where(overlap)
        var_retr_plt = var_retr.where(overlap)
        var_ds = xr.Dataset({'var_obs': var_obs_plt, 'var_retr': var_retr_plt, 'col_field': col_field})
        axparams_var = axparams[varname]
        fig, ax = PIPSplot.plot_one2one(var_ds, 'var_obs', 'var_retr', axparams_var)
        ax.set_title(title)
        plot_filename = '{}_one2one_PIPS_{}{}{}.{}'.format(varname, retr_name, ND_tag, filter_tag, figfmt)
        plot_path = os.path.join(plot_dir, plot_filename)
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')   

axparams = {
    'Dm43': {
        'var_lims': [[0.5, 4.5], [0.5, 4.5]],
        'col_field': 'col_field',
        'col_field_lims': [0.1, 200.],
        'norm': colors.LogNorm(vmin=0.1, vmax=200.),
        'alpha': 0.75,
        'markersize': 5,
        'label_x': r'$D_m$ (obs; mm)',
        'label_y': r'$D_m$ (retrieved; mm)',
        'label_cb': r'$RR$ (mm h$^{-1}$)'
    },
    'Nt': {
        'var_lims': [[0.0, 4.5], [0.0, 4.5]],
        'plot_log': [True, True],
        'col_field': 'col_field',
        'col_field_lims': [0.1, 200.],
        'norm': colors.LogNorm(vmin=0.1, vmax=200.),
        'alpha': 0.75,
        'markersize': 5,
        'label_x': r'$N_T$ (obs; #/m$^3$)',
        'label_y': r'$N_T$ (retrieved; #/m$^3$)',
        'label_cb': r'$RR$ (mm h$^{-1}$)'
    },
    'RR': {
        'var_lims': [[0.0, 250.], [0.0, 250.]],
        'plot_log': [False, False],
        'col_field': 'col_field',
        'col_field_lims': [0.1, 200.],
        'norm': colors.LogNorm(vmin=0.1, vmax=200.),
        'alpha': 0.75,
        'markersize': 5,
        'label_x': r'$RR$ (obs; mm h$^{-1}$)',
        'label_y': r'$RR$ (retrieved; mm h$^{-1}$)',
        'label_cb': r'$RR$ (mm h$^{-1}$)'
    },
    'D0': {
        'var_lims': [[0.5, 4.5], [0.5, 4.5]],
        'col_field': 'col_field',
        'col_field_lims': [0.1, 200.],
        'norm': colors.LogNorm(vmin=0.1, vmax=200.),
        'alpha': 0.75,
        'markersize': 5,
        'label_x': r'$D_0$ (obs; mm)',
        'label_y': r'$D_0$ (retrieved; mm)',
        'label_cb': r'$RR$ (mm h$^{-1}$)'
    },
    'ZH': {
        'var_lims': [[0.0, 60.], [0.0, 60.]],
        'col_field': 'col_field',
        'col_field_lims': [0.1, 200.],
        'norm': colors.LogNorm(vmin=0.1, vmax=200.),
        'alpha': 0.75,
        'markersize': 15,
        'label_x': r'$Z_H$ ({}; dBZ)',
        'label_y': r'$Z_H$ ({}; dBZ)',
        'label_cb': r'$RR$ (mm h$^{-1}$)'
    },
    'ZDR': {
        'var_lims': [[0.0, 3.], [0.0, 3.]],
        'col_field': 'col_field',
        'col_field_lims': [0.1, 200.],
        'norm': colors.LogNorm(vmin=0.1, vmax=200.),
        'alpha': 0.75,
        'markersize': 15,
        'label_x': r'$Z_{{DR}}$ ({}; dB)',
        'label_y': r'$Z_{{DR}}$ ({}; dB)',
        'label_cb': r'$RR$ (mm h$^{-1}$)'
    },
    'lambda': {
        'var_lims': [[0.0, 20.], [0.0, 20.]],
        'col_field': 'col_field',
        'col_field_lims': [0.1, 200.],
        'norm': colors.LogNorm(vmin=0.1, vmax=200.),
        'alpha': 0.75,
        'markersize': 15,
        'label_x': r'$\lambda$ ({}; mm$^{{-1}}$)',
        'label_y': r'$\lambda$ ({}; mm$^{{-1}}$)',
        'label_cb': r'$RR$ (mm h$^{-1}$)'
    },
    'mu': {
        'var_lims': [[-2.0, 30.], [-2.0, 30.]],
        'col_field': 'col_field',
        'col_field_lims': [0.1, 200.],
        'norm': colors.LogNorm(vmin=0.1, vmax=200.),
        'alpha': 0.75,
        'markersize': 15,
        'label_x': r'$\mu$ ({})',
        'label_y': r'$\mu$ ({})',
        'label_cb': r'$RR$ (mm h$^{-1}$)'
    }
}

In [ ]:
ND_tags = ['_qc', '_RB15_qc']
dD = pp.parsivel_parameters['max_diameter_bins_mm'] - pp.parsivel_parameters['min_diameter_bins_mm']

In [ ]:
# parsivel_ds = parsivel_ds_read
# Filter on RR > 0.1
# parsivel_ds = parsivel_ds_read.where(parsivel_ds_read['rainrate_derived{}'.format(ND_tags[0])] > 0.1, drop=True)
# Filter on RR > 5 and counts > 1000
parsivel_ds = parsivel_ds_read.where((parsivel_ds_read['rainrate_derived{}'.format(ND_tags[0])] > 5.0) &
                                     (parsivel_ds_read['pcount_derived{}'.format(ND_tags[0])] > 1000), drop=True)
ZH_dict = {}
ZDR_dict = {}
for ND_tag in ND_tags:
    dualpol_dict = dualpol.calpolrain(10.7, '/Users/dawson29/Projects/pyPIPS/tmatrix/S-Band/SCTT_RAIN_fw100.dat',
                                      parsivel_ds['ND{}'.format(ND_tag)], dD)
    ZH_dict[ND_tag] = dualpol_dict['REF']
    ZDR_dict[ND_tag] = dualpol_dict['ZDR']

In [ ]:
# Plot Z one-to-one scatterplots
axparams_var = axparams['ZH'].copy()
axparams_var['col_field'] = None
axparams_var['label_x'] = axparams_var['label_x'].format(ND_tags[0][1:])
axparams_var['label_y'] = axparams_var['label_y'].format(ND_tags[1][1:])
title = r'$Z_H$: {} vs. {}'.format(ND_tags[0][1:], ND_tags[1][1:])

plot_radar_field_one2one(ND_tags[0], ND_tags[1], ZH_dict[ND_tags[0]], ZH_dict[ND_tags[1]], None, title, 
                         axparams_var, deployment_name=None, min_val=0.)

In [ ]:
# Plot ZDR one-to-one scatterplots
axparams_var = axparams['ZDR'].copy()
axparams_var['col_field'] = None
axparams_var['label_x'] = axparams_var['label_x'].format(ND_tags[0][1:])
axparams_var['label_y'] = axparams_var['label_y'].format(ND_tags[1][1:])
title = r'$Z_{{DR}}$: {} vs. {}'.format(ND_tags[0][1:], ND_tags[1][1:])

plot_radar_field_one2one(ND_tags[0], ND_tags[1], ZDR_dict[ND_tags[0]], ZDR_dict[ND_tags[1]], None, title, 
                         axparams_var, deployment_name=None, min_val=0., max_val=3.)

In [ ]:
# Plot lambda one-to-one scatterplots
axparams_var = axparams['lambda'].copy()
axparams_var['col_field'] = None
axparams_var['label_x'] = axparams_var['label_x'].format(ND_tags[0][1:])
axparams_var['label_y'] = axparams_var['label_y'].format(ND_tags[1][1:])
var1 = parsivel_ds['DSD_TMM246{}'.format(ND_tags[0])].sel(parameter='lamda') / 1000.
var2 = parsivel_ds['DSD_TMM246{}'.format(ND_tags[1])].sel(parameter='lamda') / 1000.
title = r'$\lambda$: {} vs. {}'.format(ND_tags[0][1:], ND_tags[1][1:])

plot_radar_field_one2one(ND_tags[0], ND_tags[1], var1, var2, None, title, 
                         axparams_var, deployment_name=None, min_val=0., max_val=20.)

In [ ]:
# Plot mu one-to-one scatterplots
axparams_var = axparams['mu'].copy()
axparams_var['col_field'] = None
axparams_var['label_x'] = axparams_var['label_x'].format(ND_tags[0][1:])
axparams_var['label_y'] = axparams_var['label_y'].format(ND_tags[1][1:])
var1 = parsivel_ds['DSD_TMM246{}'.format(ND_tags[0])].sel(parameter='alpha')
var2 = parsivel_ds['DSD_TMM246{}'.format(ND_tags[1])].sel(parameter='alpha')
title = r'$\lambda$: {} vs. {}'.format(ND_tags[0][1:], ND_tags[1][1:])

plot_radar_field_one2one(ND_tags[0], ND_tags[1], var1, var2, None, title, 
                         axparams_var, deployment_name=None, min_val=-2., max_val=30.)

In [ ]:
# OLD CELLS BELOW

In [ ]:
# Filter on RR > 0.1 mm/h
# parsivel_ds = parsivel_ds_read.where(parsivel_ds_read['rainrate_derived{}'.format(ND_tag)] > 0.1, drop=True)
# filter_tag = ''
# Filter on RR > 5 and counts > 1000
parsivel_ds = parsivel_ds_read.where((parsivel_ds_read['rainrate_derived{}'.format(ND_tag)] > 5.0) &
                                     (parsivel_ds_read['pcount_derived{}'.format(ND_tag)] > 1000), drop=True)
filter_tag = 'RR5pcount1000'
print(parsivel_ds)
# Drop all times with DSD particle counts < 50
# parsivel_ds = parsivel_ds_read.where(parsivel_ds_read['pcount_derived_qc'] >= 50, drop=True)
# print(parsivel_ds)

In [ ]:
# Filter on ZDR > 0.1 for now (Guifu's retrieval code uses their own empirical fits for ZDR < 0.1, so we need
# to compute our own since we get weird artifacts using theirs)
ND = parsivel_ds['ND{}'.format(ND_tag)]
dD = pp.parsivel_parameters['max_diameter_bins_mm'] - pp.parsivel_parameters['min_diameter_bins_mm']
dualpol_dict = dualpol.calpolrain(10.7, '/Users/dawson29/Projects/pyPIPS/tmatrix/S-Band/SCTT_RAIN_fw100.dat',
                                  ND, dD)
ZH = dualpol_dict['ZH']
ZDR = dualpol_dict['ZDR']
parsivel_ds = parsivel_ds.where(ZDR > 0.1)

In [ ]:
# Rename some variables to make things easier. NOTE: Done! 

# for retr_name in ['SATP', 'C08', 'Z01', 'TMM_F']:
#     varname_old = 'Dm_retr_{}{}'.format(retr_name, ND_tag)
#     varname_new = 'Dm43_retr_{}{}'.format(retr_name, ND_tag)
#     parsivel_ds = parsivel_ds.rename({varname_old: varname_new})

ND = parsivel_ds['ND{}'.format(ND_tag)]
print(ND['min_diameter'])
dD = pp.parsivel_parameters['max_diameter_bins_mm'] - pp.parsivel_parameters['min_diameter_bins_mm']
dualpol_dict = dualpol.calpolrain(10.7, '/Users/dawson29/Projects/pyPIPS/tmatrix/S-Band/SCTT_RAIN_fw100.dat',
                                  ND, dD)
ZH = dualpol_dict['ZH']
ZDR = dualpol_dict['ZDR']
rho = parsivel_ds['rho']
W_obs, _ = dsd.calc_lwc_qr_from_bins(ND, rho)
W_obs = W_obs * 1000.
Dm_obs = parsivel_ds['Dm43{}'.format(ND_tag)] * 1000.
D0_obs = dsd.calc_D0_bin(ND.load()) * 1000.
for retr_name in ['SATP_TMM', 'C08', 'Z01', 'TMM_F']:
    ND_retr = parsivel_ds['ND_retr_{}{}'.format(retr_name, ND_tag)]
    parsivel_ds['D0_retr_{}{}'.format(retr_name, ND_tag)] = dsd.calc_D0_bin(ND_retr.load()) * 1000.
RR_obs = parsivel_ds['rainrate_derived{}'.format(ND_tag)]
Nt_obs = dsd.calc_NT_from_bins(parsivel_ds['ND{}'.format(ND_tag)])
    
# parsivel_ds['W_obs'] = W_obs
# W_retr_SATP = parsivel_ds['W_retr_SATP{}'.format(ND_tag)]
# W_retr_TMM_F = parsivel_ds['W_retr_TMM_F{}'.format(ND_tag)]
# W_retr_C08 = parsivel_ds['W_retr_C08{}'.format(ND_tag)]
# W_retr_Z01 = parsivel_ds['W_retr_Z01{}'.format(ND_tag)]


# Dm_retr_SATP = parsivel_ds['Dm_retr_SATP{}'.format(ND_tag)]
# Dm_retr_TMM_F = parsivel_ds['Dm_retr_TMM_F{}'.format(ND_tag)]
# Dm_retr_C08 = parsivel_ds['Dm_retr_C08{}'.format(ND_tag)]
# Dm_retr_Z01 = parsivel_ds['Dm_retr_Z01{}'.format(ND_tag)]

# Nt_retr_SATP = parsivel_ds['Nt_retr_SATP{}'.format(ND_tag)]
# Nt_retr_C08 = parsivel_ds['Nt_retr_C08{}'.format(ND_tag)]
# Nt_retr_Z01 = parsivel_ds['Nt_retr_Z01{}'.format(ND_tag)]
# Nt_retr_TMM_F = parsivel_ds['Nt_retr_TMM_F{}'.format(ND_tag)]
# RR_retr_SATP = parsivel_ds['RR_retr_SATP{}'.format(ND_tag)]
# RR_retr_C08 = parsivel_ds['RR_retr_C08{}'.format(ND_tag)]
# RR_retr_Z01 = parsivel_ds['RR_retr_Z01{}'.format(ND_tag)]
# RR_retr_TMM_F = parsivel_ds['RR_retr_TMM_F{}'.format(ND_tag)]

In [ ]:
# Dm
varname = 'Dm43'
var_obs = Dm_obs
col_field = RR_obs

plot_PIPS_retrieval_one2one(parsivel_ds, varname, var_obs, titles, retr_names, ND_tag, col_field, filter_tag)

In [ ]:
# Nt
varname = 'Nt'
var_obs = Nt_obs
col_field = RR_obs

plot_PIPS_retrieval_one2one(parsivel_ds, varname, var_obs, titles, retr_names, ND_tag, col_field, filter_tag)

In [ ]:
# RR
varname = 'RR'
var_obs = RR_obs
col_field = RR_obs

plot_PIPS_retrieval_one2one(parsivel_ds, varname, var_obs, titles, retr_names, ND_tag, col_field, filter_tag)

In [ ]:
# D0
varname = 'D0'
var_obs = D0_obs
col_field = RR_obs

plot_PIPS_retrieval_one2one(parsivel_ds, varname, var_obs, titles, retr_names, ND_tag, col_field, filter_tag)

In [ ]:
# Plot D0 vs ZDR and compute polynomial fit

overlap = (D0_obs > 0.) & (ZDR > 0.)
D0_plt = D0_obs.where(overlap)
ZDR_plt = ZDR.where(overlap)
RR_plt = RR_obs.where(overlap)

D0_ds = xr.Dataset({'D0': D0_plt, 'ZDR': ZDR_plt, 'RR_obs': RR_plt})

D0_plt_vals = D0_plt.values
ZDR_plt_vals = ZDR_plt.values

goodindices = np.isfinite(D0_plt_vals) & np.isfinite(ZDR_plt_vals)
D0_plt_vals = D0_plt_vals[goodindices]
ZDR_plt_vals = ZDR_plt_vals[goodindices]

In [ ]:

axparams = {
    'var_lims': [[0., 4.], [0., 5.]],
    'plot_log': [False, False],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$Z_{DR}$ (dB)',
    'label_y': r'$D_0 (mm)$', 
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
fig, ax = PIPSplot.plot_scatter(D0_ds, 'ZDR', 'D0', axparams)
poly_coeff, poly = dsd.calc_empirical_polyfit(ZDR_plt_vals, D0_plt_vals, order=3)
print(poly_coeff)

xx = np.linspace(0., 4.)
yy = poly(xx)
ax.plot(xx, yy, 'k')

D0_cao = 0.0436 * (xx**3.) - 0.216 * (xx**2.) + 1.076 * xx + 0.659
ax.plot(xx, D0_cao, 'b')

In [ ]:
# TODO: Update cells from here down

# Plot log(R/Zh) one-to-one plot
overlap = (RR_obs > 0.) & (RR_retr_SATP > 0.)
RR_obs_plt = RR_obs.where(overlap)
RR_retr_SATP_plt = RR_retr_SATP.where(overlap)

ZH_plt = ZH.where(ZH > 0.)
RRdZH_obs = RR_obs_plt/ZH_plt
RRdZH_retr_SATP = RR_retr_SATP_plt/ZH_plt

RRdZH_ds = xr.Dataset({'RRdZH_obs': RRdZH_obs, 'RRdZH_retr_SATP': RRdZH_retr_SATP, 'RR_obs': RR_obs_plt})

axparams = {
    'var_lims': [[-4., -1.5], [-4., -1.5]],
    'plot_log': [True, True],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$\log{R/Z_H}$ (obs)',
    'label_y': r'$\log{R/Z_H}$ (retrieved)',
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
PIPSplot.plot_one2one(RRdZH_ds, 'RRdZH_obs', 'RRdZH_retr_SATP', axparams)

In [ ]:
# Plot R/Zh vs. ZDR and compute polynomial fit
overlap = (RRdZH_obs > 0.) & (ZDR > 0.)
RRdZH_obs_plt = RRdZH_obs.where(overlap)
ZDR_plt = ZDR.where(overlap)
RR_obs_plt = RR_obs.where(overlap)

RRdZH_ZDR_ds = xr.Dataset({'RRdZH': RRdZH_obs_plt, 'ZDR': ZDR_plt, 'RR_obs': RR_obs_plt})

RRdZH_vals = RRdZH_obs_plt.values
log_RRdZH_vals = np.log10(RRdZH_vals)
ZDR_vals = ZDR_plt.values

goodindices = np.isfinite(log_RRdZH_vals) & np.isfinite(ZDR_vals)
log_RRdZH_vals = log_RRdZH_vals[goodindices]
ZDR_vals = ZDR_vals[goodindices]

In [ ]:
from scipy.optimize import curve_fit
def func(x, a, b, c, d):
    return a*x**3. + b*x**2. + c*x + d

popt, pcov = curve_fit(func, ZDR_vals, log_RRdZH_vals)
print(popt)

# Gives almost the same results (reversed coefficient order...)

In [ ]:

axparams = {
    'var_lims': [[0., 4.], [-4., -1.5]],
    'plot_log': [False, True],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$Z_{DR}$ (dB)',
    'label_y': r'$\log{R/Z_H}$', 
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
fig, ax = PIPSplot.plot_scatter(RRdZH_ZDR_ds, 'ZDR', 'RRdZH', axparams)
poly_coeff, poly = dsd.calc_empirical_polyfit(ZDR_vals, log_RRdZH_vals, order=3)
print(poly_coeff)

xx = np.linspace(0., 4.)
yy = poly(xx)
ax.plot(xx, 10.**yy, 'k')

RdZH_cao = 10.**(-0.0363 * (xx**3.) + 0.316 * (xx**2.) - 1.178 * xx - 1.964)
ax.plot(xx, RdZH_cao, 'b')

In [ ]:
# Plot W/ZH one-to-one
overlap = (W_obs > 0.) & (W_retr_SATP > 0.)
W_obs_plt = W_obs.where(overlap)
W_retr_SATP_plt = W_retr_SATP.where(overlap)
RR_obs_plt = RR_obs.where(overlap)

WdZH_obs = W_obs_plt/ZH_plt
WdZH_retr_SATP = W_retr_SATP_plt/ZH_plt
WdZH_ds = xr.Dataset({'WdZH_obs': WdZH_obs, 'WdZH_retr': WdZH_retr_SATP, 'RR_obs': RR_obs_plt})

axparams = {
    'var_lims': [[-6., -2.], [-6, -2.]],
    'plot_log': [True, True],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$\log{W/Z_H}$ (obs)',
    'label_y': r'$\log{W/Z_H}$ (retrieved)',
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
PIPSplot.plot_one2one(WdZH_ds, 'WdZH_obs', 'WdZH_retr', axparams)

In [ ]:
# Compute empirical fit for W/ZH vs ZDR and plot

overlap = (W_obs > 0.)
W_obs_plt = W_obs.where(overlap)
WdZH_obs = W_obs_plt/ZH_plt
overlap = (WdZH_obs > 0.) & (ZDR > 0.)
WdZH_obs = WdZH_obs.where(overlap)
ZDR_plt = ZDR.where(overlap)
RR_obs_plt = RR_obs.where(overlap)

WdZH_ZDR_ds = xr.Dataset({'WdZH': WdZH_obs, 'ZDR': ZDR_plt, 'RR_obs': RR_obs_plt})

WdZH_vals = WdZH_obs.values
log_WdZH_vals = np.log10(WdZH_vals)
ZDR_vals = ZDR_plt.values

goodindices = np.isfinite(log_WdZH_vals) & np.isfinite(ZDR_vals)
log_WdZH_vals = log_WdZH_vals[goodindices]
ZDR_vals = ZDR_vals[goodindices]

In [ ]:

axparams = {
    'var_lims': [[0., 4.], [-6., -2.]],
    'plot_log': [False, True],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$Z_{DR}$ (dB)',
    'label_y': r'$\log{W/Z_H}$', 
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
fig, ax = PIPSplot.plot_scatter(WdZH_ZDR_ds, 'ZDR', 'WdZH', axparams)
poly_coeff, poly = dsd.calc_empirical_polyfit(ZDR_vals, log_WdZH_vals, order=3)
print(poly_coeff)

xx = np.linspace(0., 4.)
yy = poly(xx)
ax.plot(xx, 10.**yy, 'k')
WdZH_cao = 10.**(-0.0493 * (xx**3.) + 0.430 * (xx**2.) - 1.542 * xx - 3.019)
ax.plot(xx, WdZH_cao, 'b')

In [ ]:
# Plot Nt/ZH one-to-one
overlap = (Nt_obs > 0.) & (Nt_retr_SATP > 0.)
Nt_obs_plt = Nt_obs.where(overlap)
Nt_retr_SATP_plt = Nt_retr_SATP.where(overlap)
NtdZH_obs = Nt_obs_plt/ZH_plt
NtdZH_retr_SATP = Nt_retr_SATP_plt/ZH_plt
RR_obs_plt = RR_obs.where(overlap)

NtdZH_ds = xr.Dataset({'NtdZH_obs': NtdZH_obs, 'NtdZH_retr': NtdZH_retr_SATP, 'RR_obs': RR_obs_plt})

axparams = {
    'var_lims': [[-3., 2.], [-3., 2.]],
    'plot_log': [True, True],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$\log{N_T/Z_H}$ (obs)',
    'label_y': r'$\log{N_T/Z_H}$ (retrieved)',
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
PIPSplot.plot_one2one(NtdZH_ds, 'NtdZH_obs', 'NtdZH_retr', axparams)

In [ ]:
# Plot Z vs. ZDR
dBZ = dualpol_dict['REF']
dBZ_vals = dBZ.values
ZDR_vals = ZDR.values

goodindices = np.isfinite(dBZ_vals) & np.isfinite(ZDR_vals)
dBZ_vals = dBZ_vals[goodindices]
ZDR_vals = ZDR_vals[goodindices]

ZH_ZDR_ds = xr.Dataset({'ZH': dBZ, 'ZDR': ZDR, 'RR_obs': RR_obs})

axparams = {
    'var_lims': [[0., 60.], [0., 6.]],
    'plot_log': [False, False],
    'col_field': 'RR_obs',
    'col_field_lims': [0.1, 200.],
    'norm': colors.LogNorm(vmin=0.1, vmax=200.),
    'alpha': 0.75,
    'markersize': 5,
    'label_x': r'$Z_{H}$ (dBZ)',
    'label_y': r'$Z_{DR}$ (dB)', 
    'label_cb': r'$RR$ (mm h$^{-1}$)'
}
fig, ax = PIPSplot.plot_scatter(ZH_ZDR_ds, 'ZH', 'ZDR', axparams)
ax.set_aspect('auto')
poly_coeff, poly = dsd.calc_empirical_polyfit(dBZ_vals, ZDR_vals, order=2)
print(poly_coeff)

# xx = np.linspace(0., 4.)
# yy = poly(xx)
# ax.plot(xx, 10.**yy, 'k')
# WdZH_cao = 10.**(-0.0493 * (xx**3.) + 0.430 * (xx**2.) - 1.542 * xx - 3.019)
# ax.plot(xx, WdZH_cao, 'b')